In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import *
from wordcloud import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
nltk.download('wordnet')

In [ ]:
data = pd.read_csv("fake_or_real_news.csv")
data = data.drop('Unnamed: 0', axis=1)
data = data.drop('title', axis=1)

In [ ]:
label_nums = []
for i in data['label'].values:
  if i == "FAKE":
    label_nums.append(1)
  else:
    label_nums.append(2)

data.insert(2, "label num", label_nums, True)

In [ ]:
data.head()

In [ ]:
X = data['text'].values
y = data['label num'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=2)

In [ ]:
def preprocess(data):
  stemmer = PorterStemmer()
  preprocessed_data = []

  for entry in data:
    entry = word_tokenize(entry)

    for word_index in range(len(entry)-1,-1,-1):
      if entry[word_index].lower() in STOPWORDS:
        entry.pop(word_index)
      else:
        entry[word_index] = stemmer.stem(entry[word_index])
    
    preprocessed_data.append(entry)

  return preprocessed_data

In [ ]:
train_sentences = preprocess(X_train)
train_labels = y_train

In [ ]:
test_sentences = preprocess(X_test)
test_labels = y_test

In [ ]:
train_sentences = [" ".join(t) for t in train_sentences]

train_vectorizer = CountVectorizer()

train_vectorizer.fit(train_sentences)
train_vect = train_vectorizer.transform(train_sentences).toarray()
  
model = LogisticRegression()

model.fit(train_vect, train_labels)

In [ ]:
print(train_sentences[0:5])
print([" ".join(t) for t in train_sentences[0:5]])
print(train_labels[0:5])
print([l for l in train_labels[0:5]])

In [ ]:
test_sentences = [" ".join(t) for t in  test_sentences]
  
test_vect = train_vectorizer.transform(test_sentences).toarray()

preds = model.predict(test_vect)
  
acc = metrics.accuracy_score(test_labels, preds)

print(acc)